In [1]:
import polars as pl

In [2]:
regions_df = pl.read_csv("../data/continents2.csv", infer_schema_length=0)
main_df = pl.read_csv("../data/WorldSustainabilityDataset.csv", infer_schema_length=0)

In [3]:
regions_df.columns

['name',
 'alpha-2',
 'alpha-3',
 'country-code',
 'iso_3166-2',
 'region',
 'sub-region',
 'intermediate-region',
 'region-code',
 'sub-region-code',
 'intermediate-region-code']

In [4]:
main_df.columns

['Country Name',
 'Country Code',
 'Year',
 'Access to electricity (% of population) - EG.ELC.ACCS.ZS',
 'Adjusted net national income per capita (annual % growth) - NY.ADJ.NNTY.PC.KD.ZG',
 'Adjusted net savings, excluding particulate emission damage (% of GNI) - NY.ADJ.SVNX.GN.ZS',
 'Adjusted savings: carbon dioxide damage (% of GNI) - NY.ADJ.DCO2.GN.ZS',
 'Adjusted savings: natural resources depletion (% of GNI) - NY.ADJ.DRES.GN.ZS',
 'Adjusted savings: net forest depletion (% of GNI) - NY.ADJ.DFOR.GN.ZS',
 'Adjusted savings: particulate emission damage (% of GNI) - NY.ADJ.DPEM.GN.ZS',
 'Automated teller machines (ATMs) (per 100,000 adults) - FB.ATM.TOTL.P5',
 'Broad money (% of GDP) - FM.LBL.BMNY.GD.ZS',
 'Children out of school (% of primary school age) - SE.PRM.UNER.ZS',
 'Compulsory education, duration (years) - SE.COM.DURS',
 'Cost of business start-up procedures, female (% of GNI per capita) - IC.REG.COST.PC.FE.ZS',
 'Cost of business start-up procedures, male (% of GNI per cap

In [5]:
merged = main_df.join(
    regions_df,
    left_on="Country Name",
    right_on="name",
    how="left"
)

In [6]:
not_matched = merged.filter(
    pl.col("sub-region").is_null()
)['Country Name'].unique().to_list()

print(len(not_matched))
not_matched

23


['North Macedonia',
 'St. Lucia',
 'Bahamas, The',
 'Guinea-Bissau',
 'Lao PDR',
 'Macao SAR, China',
 'Slovak Republic',
 'Iran, Islamic Rep.',
 'Egypt, Arab Rep.',
 'West Bank and Gaza',
 "Korea, Dem. People's Rep.",
 'Gambia, The',
 'Congo, Dem. Rep.',
 'Hong Kong SAR, China',
 'Syrian Arab Republic',
 'Congo, Rep.',
 'Kyrgyz Republic',
 'Russian Federation',
 "Cote d'Ivoire",
 'St. Vincent and the Grenadines',
 'Korea, Rep.',
 'Venezuela, RB',
 'Bosnia and Herzegovina']

### 23 countries could not be matched; let's investigate

In [7]:
regions_df.filter(
    pl.col('name').str.contains('Kyrgyz')
)

name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
str,str,str,str,str,str,str,str,str,str,str
"""Kyrgyzstan""","""KG""","""KGZ""","""417""","""ISO 3166-2:KG""","""Asia""","""Central Asia""",null,"""142""","""143""",null


In [8]:
main_df.join(
    regions_df,
    left_on="Country Code",
    right_on="alpha-3",
    how="left"
).filter(
    pl.col("sub-region").is_null()
)['Country Name'].unique().to_list()

[]

### Use country code and alpha-3 to merge instead

In [9]:
merged = main_df.join(
    regions_df,
    left_on="Country Code",
    right_on="alpha-3",
    how="left"
)

In [10]:
merged.unique(subset='Country Name').sample(5)

Country Name,Country Code,Year,Access to electricity (% of population) - EG.ELC.ACCS.ZS,Adjusted net national income per capita (annual % growth) - NY.ADJ.NNTY.PC.KD.ZG,"Adjusted net savings, excluding particulate emission damage (% of GNI) - NY.ADJ.SVNX.GN.ZS",Adjusted savings: carbon dioxide damage (% of GNI) - NY.ADJ.DCO2.GN.ZS,Adjusted savings: natural resources depletion (% of GNI) - NY.ADJ.DRES.GN.ZS,Adjusted savings: net forest depletion (% of GNI) - NY.ADJ.DFOR.GN.ZS,Adjusted savings: particulate emission damage (% of GNI) - NY.ADJ.DPEM.GN.ZS,"Automated teller machines (ATMs) (per 100,000 adults) - FB.ATM.TOTL.P5",Broad money (% of GDP) - FM.LBL.BMNY.GD.ZS,Children out of school (% of primary school age) - SE.PRM.UNER.ZS,"Compulsory education, duration (years) - SE.COM.DURS","Cost of business start-up procedures, female (% of GNI per capita) - IC.REG.COST.PC.FE.ZS","Cost of business start-up procedures, male (% of GNI per capita) - IC.REG.COST.PC.MA.ZS",Exports of goods and services (% of GDP) - NE.EXP.GNFS.ZS,Final consumption expenditure (% of GDP) - NE.CON.TOTL.ZS,GDP (current US$) - NY.GDP.MKTP.CD,GDP per capita (current US$) - NY.GDP.PCAP.CD,General government final consumption expenditure (% of GDP) - NE.CON.GOVT.ZS,Gross national expenditure (% of GDP) - NE.DAB.TOTL.ZS,Gross savings (% of GDP) - NY.GNS.ICTR.ZS,Imports of goods and services (% of GDP) - NE.IMP.GNFS.ZS,"Inflation, consumer prices (annual %) - FP.CPI.TOTL.ZG","Primary completion rate, total (% of relevant age group) - SE.PRM.CMPT.ZS",Proportion of seats held by women in national parliaments (%) - SG.GEN.PARL.ZS,"Pupil-teacher ratio, primary - SE.PRM.ENRL.TC.ZS",Renewable electricity output (% of total electricity output) - EG.ELC.RNEW.ZS,Renewable energy consumption (% of total final energy consumption) - EG.FEC.RNEW.ZS,"School enrollment, preprimary (% gross) - SE.PRE.ENRR","School enrollment, primary (% gross) - SE.PRM.ENRR","School enrollment, secondary (% gross) - SE.SEC.ENRR",Trade (% of GDP) - NE.TRD.GNFS.ZS,Women Business and the Law Index Score (scale 1-100) - SG.LAW.INDX,Prevalence of undernourishment (%) - SN_ITK_DEFC - 2.1.1,Proportion of population below international poverty line (%) - SI_POV_DAY1 - 1.1.1,Proportion of population covered by at least a 2G mobile network (%) - IT_MOB_2GNTWK - 9.c.1,Proportion of population covered by at least a 3G mobile network (%) - IT_MOB_3GNTWK - 9.c.1,Proportion of population using basic drinking water services (%) - SP_ACS_BSRVH2O - 1.4.1,"Unemployment rate, male (%) - SL_TLF_UEM - 8.5.2","Unemployment rate, women (%) - SL_TLF_UEM - 8.5.2","Annual production-based emissions of carbon dioxide (CO2), measured in million tonnes",Continent,Gini index (World Bank estimate) - SI.POV.GINI,Income Classification (World Bank Definition),Individuals using the Internet (% of population) - IT.NET.USER.ZS,"Life expectancy at birth, total (years) - SP.DYN.LE00.IN","Population, total - SP.POP.TOTL",Regime Type (RoW Measure Definition),Rural population (% of total population) - SP.RUR.TOTL.ZS,Total natural resources rents (% of GDP) - NY.GDP.TOTL.RT.ZS,Urban population (% of total population) - SP.URB.TOTL.IN.ZS,World Regions (UN SDG Definition),name,alpha-2,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Albania""","""ALB""","""2000""","""100""",null,"""16.02109255""","""1.563257993""","""0.292709162""","""0""","""0.455814504""",null,"""65.46317162""","""4.616650105""","""8""",null,null,"""20.11536376""","""92.42716152""","""3480355189""","""1126.683318""","""9.69235792""","""123.2234207""","""25.94104928""","""43.33887425""","""0.050018136""","""96.58268738""","""5.161290323""","""22.56784""",

In [11]:
merged.write_csv("../data/sustainability_w_regions.csv")